In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


In [19]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(1,8,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(8,16,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc = nn.Sequential(
        nn.Linear(16*7*7,128),
        nn.ReLU(),
        nn.Linear(128,10)
    )
  def forward(self, x):
    x = self.conv(x)
    x = x.view(x.size(0),-1)
    x = self.fc(x)
    return x

In [49]:
!unzip test.zip
!unzip train.zip

Archive:  test.zip
replace test/test_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
folder = "./train/*.jpg"
paths = sorted(glob.glob("./train/*.jpg"))
imgs = []
for path in paths:
    img = Image.open(path)
    imgs.append(img)
X = np.array(imgs).astype(np.float32)
X = X.reshape(-1, 1, 28, 28)
print(X.shape)

(60000, 1, 28, 28)


In [9]:

X /= 255
np.max(X)

np.float32(1.0)

In [10]:
y = pd.read_csv("./train_master.csv")
y = y.sort_values("file_name")
y = y["category_id"].values.astype(np.int64)
y = y.reshape(-1)
print(y.shape)

(60000,)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [15]:
X_train_t = torch.from_numpy(X_train)
y_train_t = torch.from_numpy(y_train)
X_test_t  = torch.from_numpy(X_test)
y_test_t  = torch.from_numpy(y_test)

In [17]:
batch_size = 128
train_ds = TensorDataset(X_train_t, y_train_t)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_ds = TensorDataset(X_test_t, y_test_t)
test_dl = DataLoader(test_ds,batch_size=batch_size, shuffle=True)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [22]:
def train_cnn(num_epochs=10):
  for epoch in range(1,num_epochs + 1):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for X_batch, y_batch in train_dl:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      optimizer.zero_grad()
      outputs = model(X_batch)
      loss = criterion(outputs, y_batch)
      loss.backward()
      optimizer.step()
      running_loss += loss.item() * X_batch.size(0)
      _, predicted = torch.max(outputs.data, 1)
      total += y_batch.size(0)
      correct += (predicted == y_batch).sum().item()
    train_loss = running_loss / total
    train_acc = correct / total

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
      for X_batch, y_batch in test_dl:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
    test_acc = correct / total
    print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
  return model

In [23]:
cnn_model = train_cnn(num_epochs=10)

Epoch 1, Train Loss: 0.8916, Train Acc: 0.7172, Test Acc: 0.9332
Epoch 2, Train Loss: 0.1463, Train Acc: 0.9561, Test Acc: 0.9656
Epoch 3, Train Loss: 0.0966, Train Acc: 0.9708, Test Acc: 0.9722
Epoch 4, Train Loss: 0.0783, Train Acc: 0.9754, Test Acc: 0.9748
Epoch 5, Train Loss: 0.0639, Train Acc: 0.9806, Test Acc: 0.9792
Epoch 6, Train Loss: 0.0540, Train Acc: 0.9829, Test Acc: 0.9772
Epoch 7, Train Loss: 0.0487, Train Acc: 0.9849, Test Acc: 0.9842
Epoch 8, Train Loss: 0.0431, Train Acc: 0.9860, Test Acc: 0.9839
Epoch 9, Train Loss: 0.0384, Train Acc: 0.9878, Test Acc: 0.9823
Epoch 10, Train Loss: 0.0343, Train Acc: 0.9886, Test Acc: 0.9861
